In [2]:
import os
os.chdir('/Users/aaron/Desktop/MAC_paper/mac_evo/')

In [ ]:
import glob
import pandas as pd

ska_build = 'ska build -f ska/file_list_{x}.txt -k 31 -o ska/ska_index_{x} --threads 4'
ska_align = 'ska align --min-freq 1 --filter no-filter ska/ska_index_{x}.skf -o ska/ska_alignment_{x}.aln --ambig-mask --threads 4'
ska_map = 'ska map -o ska/ska_alignment_{x}.aln --ambig-mask --threads 4 databases/GCF_009741445.1/GCF_009741445.1_ASM974144v1_genomic.fna ska/ska_index_{x}.skf'
run_gubbins = 'run_gubbins.py --prefix ska/gubbins_{x} ska/ska_alignment_{x}.aln --threads 12'
snp_sites = f'docker run -v $PWD:/data staphb/snp-sites snp-sites -c {out_dir}/gubbins.filtered_polymorphic_sites.fasta > {out_dir}/gubbins.core.aln'
snp_dists = f'docker run -v $PWD:/data staphb/snp-dists snp-dists {out_dir}/gubbins.core.aln > {out_dir}/snp_dists.txt'

df = pd.read_csv('data/fastbaps.tsv', sep='\t')
clusters = [1, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18]

cmds = []
for cluster in clusters:
    name = 'fastbaps_cluster_' + str(cluster).zfill(2)
    target = f'ska/gubbins_{name}.filtered_polymorphic_sites.fasta'
    if not os.path.exists(target):
        isolates = df[df['level_1'] == cluster]['isolate'].to_list()
        output = file_list[file_list['isolate'].isin(isolates)]
        output.to_csv(f'ska/file_list_{name}.txt', sep='\t', index=False, header=False)
        ## run the ska build command
        build = ska_build.format(x=name)
        map = ska_map.format(x=name)
        gubbins = run_gubbins.format(x=name)
        cmd = f'{build} && {map} && {gubbins}'
        cmds.append(cmd)
# write the commands to a file
with open('ska_commands.sh', 'w') as f:
    for cmd in cmds:
        f.write(cmd + '\n')


In [31]:
cids = [2, 3, 4, 5, 9, 10, 11, 18]
with open('tidy_ska.sh', 'w') as f:
    for cid in cids:
        name = 'fastbaps_cluster_' + str(cid).zfill(2)
        cmd = f'rm ska/*{name}*'
        f.write(cmd + '\n')


In [33]:
snp_sites = 'docker run -v $PWD:/data staphb/snp-sites snp-sites -c ska/gubbins_{x}.filtered_polymorphic_sites.fasta > ska/core_{x}.aln'
snp_dists = 'docker run -v $PWD:/data staphb/snp-dists snp-dists ska/core_{x}.aln > ska/core_{x}.dist'

with open('run_snp_dists.sh', 'w') as f:
    for cluster in clusters:
        name = 'fastbaps_cluster_' + str(cluster).zfill(2)
        target = f'ska/core_{name}.aln'
        if not os.path.exists(target):
            cmd_1 = snp_sites.format(x=name)
            cmd_2 = snp_dists.format(x=name)
            cmd = f'{cmd_1} && {cmd_2}'
            f.write(cmd + '\n')

In [5]:
import glob
import os

os.chdir('/Users/aaron/Desktop/MAC_paper/mac_evo/')
os.listdir('ska')

fastas = glob.glob('ska/*.fasta')

with open('file_list.txt', 'w') as f:
    for fasta in fastas:
        name = fasta.split('/')[-1].split('.')[0]
        f.write(f'{name}\t{fasta}\n')

In [ ]:
## try ska with fastq files
import glob
import pandas as pd

ska_build = 'ska build -f ska/file_list.txt -k 31 -o ska/index --threads 12'
ska_align = 'ska align --min-freq 1 --filter no-filter ska_fq/ska_index_{x}.skf -o ska_fq/ska_alignment_{x}.aln --ambig-mask --threads 4'
ska_map = 'ska map -o ska_fq/ska_alignment_{x}.aln --ambig-mask --threads 4 databases/GCF_009741445.1/GCF_009741445.1_ASM974144v1_genomic.fna ska_fq/ska_index_{x}.skf'
run_gubbins = 'run_gubbins.py --prefix ska_fq/gubbins_{x} ska_fq/ska_alignment_{x}.aln --threads 12'

df = pd.read_csv('data/metadata.tsv', sep='\t')
isolates = df['isolate'].to_list()

if not os.path.exists('ska_fq'):
    os.makedirs('ska_fq')

with open('ska_fq/file_list.txt', 'w') as f:
    for isolate in isolates:
        reads = sorted(glob.glob('reads/processed/{}*'.format(isolate)))
        output = f'{isolate}\t{reads[0]}\t{reads[1]}\n'
        f.write(output)

df = pd.read_csv('data/fastbaps.tsv', sep='\t')
clusters = [1, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18]

file_list = pd.read_csv('ska_fq/file_list.txt', sep='\t', header=None, names=['isolate', 'R1', 'R2'])

cmds = []
for cluster in clusters:
    name = 'fastbaps_cluster_' + str(cluster).zfill(2)
    target = f'ska_fq/gubbins_{name}.filtered_polymorphic_sites.fasta'
    if not os.path.exists(target):
        isolates = df[df['level_1'] == cluster]['isolate'].to_list()
        output = file_list[file_list['isolate'].isin(isolates)]
        output.to_csv(f'ska_fq/file_list_{name}.txt', sep='\t', index=False, header=False)
        ## run the ska build command
        build = ska_build.format(x=name)
        map = ska_map.format(x=name)
        gubbins = run_gubbins.format(x=name)
        cmd = f'{build} && {map} && {gubbins}'
        cmds.append(cmd)
# write the commands to a file
with open('ska_fq_commands.sh', 'w') as f:
    for cmd in cmds:
        f.write(cmd + '\n')


In [ ]:
df = pd.read_csv('data/metadata.tsv', sep='\t')
isolates = df['isolate'].to_list()

if not os.path.exists('ska_fq'):
    os.makedirs('ska_fq')

with open('ska_fq/file_list.txt', 'w') as f:
    for isolate in isolates:
        reads = sorted(glob.glob('reads/processed/{}*'.format(isolate)))
        output = f'{isolate}\t{reads[0]}\t{reads[1]}\n'
        f.write(output)

clusters = [1, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18]

cmds = []
for cluster in clusters:
    name = 'fastbaps_cluster_' + str(cluster).zfill(2)
    target = f'ska_fq/gubbins_{name}.filtered_polymorphic_sites.fasta'
    if not os.path.exists(target):
        isolates = df[df['level_1'] == cluster]['isolate'].to_list()
        output = file_list[file_list['isolate'].isin(isolates)]
        output.to_csv(f'ska_fq/file_list_{name}.txt', sep='\t', index=False, header=False)
        ## run the ska build command
        build = ska_build.format(x=name)
        map = ska_map.format(x=name)
        gubbins = run_gubbins.format(x=name)
        cmd = f'{build} && {map} && {gubbins}'
        cmds.append(cmd)
# write the commands to a file
with open('ska_fq_commands.sh', 'w') as f:
    for cmd in cmds:
        f.write(cmd + '\n')


In [24]:
snp_sites = 'docker run -v $PWD:/data staphb/snp-sites snp-sites -c ska_fq/gubbins_{x}.filtered_polymorphic_sites.fasta > ska_fq/core_{x}.aln'
snp_dists = 'docker run -v $PWD:/data staphb/snp-dists snp-dists ska_fq/core_{x}.aln > ska_fq/core_{x}.dist'

with open('run_snp_dists.sh', 'w') as f:
    for cluster in clusters:
        name = 'fastbaps_cluster_' + str(cluster).zfill(2)
        target = f'ska_fq/core_{name}.aln'
        if not os.path.exists(target):
            cmd_1 = snp_sites.format(x=name)
            cmd_2 = snp_dists.format(x=name)
            cmd = f'{cmd_1} && {cmd_2}'
            f.write(cmd + '\n')

In [ ]:
from Bio import SeqIO
import os
from pathlib import Path

ska_build = 'ska build -f snippy_x_fastbaps/ska/file_list.txt -k 31 -o snippy_x_fastbaps/ska/core --threads 4'
ska_align = 'ska align --min-freq 1 --filter no-filter snippy_x_fastbaps/ska/core.skf -o snippy_x_fastbaps/ska/core.aln --ambig-mask --threads 4'

run_gubbins = 'run_gubbins.py --prefix ska_fq/gubbins_{x} ska_fq/ska_alignment_{x}.aln --threads 12'


os.chdir('/Users/aaron/Desktop/MAC_paper/mac_evo/')

if not os.path.exists('snippy_x_fastbaps/ska'):
    os.makedirs('snippy_x_fastbaps/ska')

fastas = [x.as_posix() for x in Path('snippy_x_fastbaps').rglob('*.fasta')]
for fasta in fastas:
    for record in SeqIO.parse(fasta, "fasta"):
        isolate = record.id
        if not 'Reference' in isolate:
            sequence = str(record.seq)
            with open(f'snippy_x_fastbaps/ska/{isolate}.fasta', 'w') as f:
                f.write(f'>{isolate}\n{sequence}\n')

files = [x.as_posix() for x in Path('snippy_x_fastbaps/ska').glob('*.fasta')]
with open('snippy_x_fastbaps/ska/file_list.txt', 'w') as f:
    for file in files[0:10]:
        isolate = file.split('/')[-1].split('.')[0]
        f.write(f'{isolate}\t{file}\n')

with open('run_ska.sh', 'w') as f:
    cmd = f'{ska_build} && {ska_align}\n'
    f.write(cmd)

In [ ]:
## try ska with fastq files
import glob
import os
import pandas as pd

os.chdir('/Users/aaron/Desktop/MAC_paper/mac_evo/')

if not os.path.exists('ska'):
    os.makedirs('ska')

ska_build = 'ska build -f ska/file_list.txt -k 31 -o ska/index --threads 12 -v'
ska_align = 'ska align --min-freq 1 --filter no-filter ska/index.skf -o ska/alignment.aln --ambig-mask --threads 12 -v'
fastbaps = 'Rscript src/run_fastbaps.R && Rscript src/list_fastbaps_clusters.R'

with open('ska/file_list.txt', 'w') as f:
    assemblies = [x for x in glob.glob('assemblies/*') if not 'Reference' in x]
    for assembly in assemblies:
        isolate = os.path.basename(assembly.split('.')[0])
        reads = '\t'.join(sorted(glob.glob(f'reads/processed/{isolate}*')))
        row = f'{isolate}\t{reads}\n'
        f.write(row)

with open('run_ska.sh', 'w') as f:
    f.write(ska_build + '\n')
    f.write(ska_align + '\n')
    f.write(fastbaps + '\n')


In [5]:
import glob
import os
import pandas as pd

ska_build = 'ska build -f {out_dir}/file_list.txt -k 31 -o {out_dir}/index --threads {threads} -v'
ska_map = 'ska map -o {out_dir}/alignment.aln --ambig-mask --threads {threads} {ref} {out_dir}/index.skf'
run_gubbins = 'run_gubbins.py --prefix {out_dir}/gubbins {out_dir}/alignment.aln --threads {threads}'
snp_sites = 'docker run -v $PWD:/data staphb/snp-sites snp-sites -c {out_dir}/gubbins.filtered_polymorphic_sites.fasta > {out_dir}/gubbins.core.aln'
snp_dists = 'docker run -v $PWD:/data staphb/snp-dists snp-dists {out_dir}/gubbins.core.aln > {out_dir}/snp_dists.txt'

def run_ska_x_fastbaps(out_dir='ska_x_fastbaps', threads=os.cpu_count()):
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    dat = pd.read_csv('data/fastbaps.tsv', sep='\t')
    clusters = [int(x) for x in open('data/fastbaps_clusters.txt', 'r').read().splitlines()]
    cmds = []
    for cluster in clusters:
        name = 'fastbaps_cluster_' + str(cluster).zfill(2)
        outdir = f'{out_dir}/{name}'
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        isolates = sorted(dat[dat['level_1'] == int(cluster)]['isolate'].to_list())
        targets = [f'{outdir}/index.skf',
                   f'{outdir}/alignment.aln',
                   f'{outdir}/gubbins.filtered_polymorphic_sites.fasta',
                   f'{outdir}/gubbins.core.aln',
                   f'{outdir}/snp_dists.txt']
        if not os.path.exists(targets[0]):
            with open(f'{outdir}/file_list.txt', 'w') as file_list:
                for isolate in isolates:
                    reads = '\t'.join(sorted(glob.glob(f'reads/processed/{isolate}*')))
                    row = f'{isolate}\t{reads}\n'
                    file_list.write(row)
            skabuild = ska_build.format(out_dir=outdir, threads=threads)
            cmds.append(skabuild)
        if not os.path.exists(targets[1]):
            ref = 'assemblies/{}.contigs.fasta'.format(isolates[0])
            skamap = ska_map.format(out_dir=outdir, threads=threads, ref=ref)
            cmds.append(skamap)
        if not os.path.exists(targets[2]):
            rungubbins = run_gubbins.format(out_dir=outdir, threads=threads)
            cmds.append(rungubbins)
        if not os.path.exists(targets[3]):
            snpsites = snp_sites.format(out_dir=outdir)
            cmds.append(snpsites)
        if not os.path.exists(targets[4]):
            snpdists = snp_dists.format(out_dir=outdir)
            cmds.append(snpdists)
    with open('run_ska_x_fastbaps.sh', 'w') as f:
        [f.write(cmd + '\n') for cmd in cmds]

run_ska_x_fastbaps()

In [2]:
import os
os.chdir('/Users/aaron/Desktop/MAC_paper/mac_evo/')